In [ ]:
import re
import os
import sys
import time
import numpy as np
import pandas as pd
import librosa_trans
from datetime import datetime
from IPython.display import clear_output, display

label

In [ ]:
data = pd.DataFrame()
sound_path_1 = os.path.abspath(r'/media/chiahong/3363-3234/Q2')         #正常
sound_path = os.path.abspath(r'/media/chiahong/3363-3234/Q2')           #外敵
test_list1 = ['Q2失王前']                               #正常
test_list2 = ['Q2失王後']                               #外敵
for test in test_list1:
    dir_path = sorted(os.listdir(os.path.join(sound_path_1, test)))
    for file in iter(dir_path):
            pattern = r'(\d{4})\.(\d{2})\.(\d{2})\.(\d{2})\.(\d{2})\.(\d{2})\.wav'
            match = re.search(pattern, file)
            if match:
                year = match.group(1)
                month = match.group(2)
                day = match.group(3)
                hour = match.group(4)
                minute = match.group(5)
                second = match.group(6)
                if int(day) < 3 or int(day) > 27:
                    data=data.append({'path':str(os.path.join(sound_path_1,test,file)),'id':test,'district':'North','state':'Normal'}, ignore_index=True)

for test in test_list2:
    dir_path = sorted(os.listdir(os.path.join(sound_path, test)))
    for file in iter(dir_path):
            pattern = r'(\d{4})\.(\d{2})\.(\d{2})\.(\d{2})\.(\d{2})\.(\d{2})\.wav'
            match = re.search(pattern, file)
            if match:
                year = match.group(1)
                month = match.group(2)
                day = match.group(3)
                hour = match.group(4)
                minute = match.group(5)
                second = match.group(6)
                if int(day) > 3:
                    data=data.append({'path':str(os.path.join(sound_path,test,file)),'id':test,'district':'North','state':'Queenless'}, ignore_index=True)

data.to_csv(r'/home/chiahong/Documents/chihli_bee_sound/2024_rpi_vs_esp_Queenless/feature_extraction/Q2.csv', index_label=None)

metadata = pd.read_csv(r'/home/chiahong/Documents/chihli_bee_sound/2024_rpi_vs_esp_Queenless/feature_extraction/Q2.csv')

# Examine dataframe
print("Metadata length:" ,len(metadata))
# metadata.tail()

print(metadata.to_string())

mfcc

In [ ]:
features = []  # 存儲MFCC特徵
labels = []    # 存儲標籤
frames_max = 0  # 最大幀數

# 定義重試參數
max_retries = 5  # 每個文件的最大重試次數
retry_delay = 5  # 重試間的延遲時間（秒）

# 遍歷每一行元數據
for index, row in metadata.iloc[0:].iterrows():

    # 提取文件路徑、colony label 和 state label
    file_path = os.path.abspath(str(row["path"]))
    colony_label = row["id"]
    state_label = row["state"]

    # 使用重試機制提取MFCCs
    retry_attempt = 0
    while retry_attempt < max_retries:
        try:
            # 提取MFCCs
            mfccs = librosa_trans.mfcc_transform(file_path)

            # 保存當前幀數
            num_frames = mfccs.shape[1]

            # 添加行（特徵/標籤）
            features.append(mfccs)
            labels.append(state_label)

            # 更新最大幀數
            if num_frames > frames_max:
                frames_max = num_frames

            # 成功 - 跳出重試循環
            print(f"從 {file_path} 成功提取特徵")
            break

        except Exception as e:  # 捕獲任何異常
            print(f"從 {file_path} 提取特徵失敗（第 {retry_attempt + 1}/{max_retries} 次）: {e}")

            # 如果嘗試次數尚未達到限制，則重試
            if retry_attempt < max_retries - 1:
                time.sleep(retry_delay)  # 延遲後重試
                retry_attempt += 1
            else:
                # 最大重試次數已達到 - 跳過到下一個文件
                print(f"{file_path} 的最大重試次數已達到。跳過到下一個文件。")
                break

    # 清除輸出並打印進度
    clear_output(wait=True)
    print("進度：{}/{}".format(index, len(metadata)))

# 對特徵進行填充，使其達到最大幀長度
padded = []
mfcc_max_padding = frames_max
for j in range(len(features)):
    size = len(features[j][0])
    if size < mfcc_max_padding:
        pad_width = mfcc_max_padding - size
        px = np.pad(features[j],
                    pad_width=((0, 0), (0, pad_width)),
                    mode='constant', constant_values=(0,))
    else:
        px = features[j]

    padded.append(px)

# 將特徵和標籤轉換為NumPy數組
X = np.array(padded)
y = np.array(labels)

# 可選地將特徵保存到磁盤
np.save(r'/home/chiahong/Documents/chihli_bee_sound/2024_rpi_vs_esp_Queenless/feature_extraction/Q3_rpi.csv_x', X)
np.save(r'/home/chiahong/Documents/chihli_bee_sound/2024_rpi_vs_esp_Queenless/feature_extraction/Q3_rpi.csv_y', y)
